In [30]:
# car_rental.py
class Car:
    def __init__(self, car_id, model, rental_price):
        self.car_id = car_id
        self.model = model
        self.rental_price = rental_price
        self.available = True
    def rent(self):
        """Mark the car as rented if it is available."""
        if not self.available:
            raise Exception("Car is not available")
        self.available = False
    def return_car(self):
        """Mark the car as available if it is currently rented."""
        if self.available:
            raise Exception("Car is already available")
        self.available = True

class CarRentalSystem:
    def __init__(self):
        # Use a dictionary to manage cars by their unique car_id.
        self.cars = {}
    def add_car(self, car: Car):
        """Add a new car to the system."""
        if car.car_id in self.cars:
            raise Exception("Car already exists")
        self.cars[car.car_id] = car
    def remove_car(self, car_id):
        """Remove a car from the system."""
        if car_id not in self.cars:
            raise Exception("Car not found")
        del self.cars[car_id]
    def rent_car(self, car_id):
        """Rent a car by its id."""
        if car_id not in self.cars:
            raise Exception("Car not found")
        car = self.cars[car_id]
        car.rent()
        return car
    def return_car(self, car_id):
        """Return a rented car by its id."""
        if car_id not in self.cars:
            raise Exception("Car not found")
        car = self.cars[car_id]
        car.return_car()
        return car
    def available_cars(self):
        """Return a list of all available cars."""
        return [car for car in self.cars.values() if car.available]

### Test Cases

In [31]:
# test_car_rental.py

import unittest
import time

### A =  → Unit Test

In [32]:
class TestCar(unittest.TestCase):
    def setUp(self):
        self.car = Car(car_id=1, model="Toyota Camry", rental_price=50)
    def test_rent_car(self):
        self.assertTrue(self.car.available)
        self.car.rent()
        self.assertFalse(self.car.available)
    def test_return_car(self):
        self.car.rent()
        self.car.return_car()
        self.assertTrue(self.car.available)
    def test_rent_unavailable_car(self):
        self.car.rent()
        with self.assertRaises(Exception) as context:
            self.car.rent()
        self.assertIn("Car is not available", str(context.exception))
    def test_return_already_available_car(self):
        with self.assertRaises(Exception) as context:
            self.car.return_car()
        self.assertIn("Car is already available", str(context.exception))
     
class TestCarRentalSystem(unittest.TestCase):
    def setUp(self):
        self.system = CarRentalSystem()
        self.car1 = Car(car_id=1, model="Toyota Camry", rental_price=50)
        self.car2 = Car(car_id=2, model="Honda Accord", rental_price=55)
        self.system.add_car(self.car1)
        self.system.add_car(self.car2)
    def test_add_car(self):
        with self.assertRaises(Exception):
            self.system.add_car(self.car1)
    def test_remove_car(self):
        self.system.remove_car(self.car1.car_id)
        with self.assertRaises(Exception):
            self.system.rent_car(self.car1.car_id)
    def test_rent_car(self):
        rented_car = self.system.rent_car(self.car1.car_id)
        self.assertFalse(rented_car.available)
    def test_return_car(self):
        self.system.rent_car(self.car2.car_id)
        returned_car = self.system.return_car(self.car2.car_id)
        self.assertTrue(returned_car.available)
    def test_available_cars(self):
        self.system.rent_car(self.car1.car_id)
        available = self.system.available_cars()
        self.assertEqual(len(available), 1)
        self.assertEqual(available[0].car_id, self.car2.car_id)

### B =  → Regression Test

In [33]:
class TestB(unittest.TestCase):
    def setUp(self):
        self.system = CarRentalSystem()
        self.car = Car(car_id=3, model="Ford Mustang", rental_price=100)
        self.system.add_car(self.car)
    def test_B_rent_already_rented_car(self):
        self.system.rent_car(self.car.car_id)
        with self.assertRaises(Exception) as context:
            self.system.rent_car(self.car.car_id)
        self.assertIn("Car is not available", str(context.exception))

### C =  → Integration Test

In [34]:
class TestC(unittest.TestCase):
    def test_full_workflow(self):
        system = CarRentalSystem()
        car1 = Car(car_id=1, model="Toyota Camry", rental_price=50)
        car2 = Car(car_id=2, model="Honda Accord", rental_price=55)
        car3 = Car(car_id=3, model="Ford Focus", rental_price=45)
        system.add_car(car1)
        system.add_car(car2)
        system.add_car(car3)
        system.rent_car(car1.car_id)
        available = system.available_cars()
        self.assertEqual(len(available), 2)
        system.return_car(car1.car_id)
        available = system.available_cars()
        self.assertEqual(len(available), 3)
        system.remove_car(car2.car_id)
        available = system.available_cars()
        self.assertEqual(len(available), 2)

### D =  → Functional Test

In [35]:
class TestD(unittest.TestCase):
    def test_user_workflow(self):
        system = CarRentalSystem()
        car = Car(car_id=10, model="Chevrolet Malibu", rental_price=60)
        system.add_car(car)
        self.assertEqual(len(system.available_cars()), 1)
        system.rent_car(car.car_id)
        self.assertEqual(len(system.available_cars()), 0)
        system.return_car(car.car_id)
        self.assertEqual(len(system.available_cars()), 1)

### E =  → Performance Test

In [36]:
class TestE(unittest.TestCase):
    def test_add_cars_(self):
        system = CarRentalSystem()
        num_cars = 10000
        start_time = time.time()
        for i in range(num_cars):
            system.add_car(Car(i, f"Model {i}", 50))
        end_time = time.time()
        elapsed = end_time - start_time
        self.assertLess(elapsed, 1, f"Adding {num_cars} cars took too long: {elapsed:.2f} seconds")

### F =  → Stress Test

In [37]:
class TestF(unittest.TestCase):
    def test_rent_return_(self):
        system = CarRentalSystem()
        num_cars = 5000
        for i in range(num_cars):
            system.add_car(Car(i, f"Model {i}", 50))
        for i in range(num_cars):
            system.rent_car(i)
        self.assertEqual(len(system.available_cars()), 0)
        for i in range(num_cars):
            system.return_car(i)
        self.assertEqual(len(system.available_cars()), num_cars)

In [38]:
# Run the tests using unittest in Jupyter Notebook
def run_tests():
    # Create a test suite and add the test cases
    suite = unittest.TestLoader().loadTestsFromTestCase(TestF)

    # TestA
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestCar))
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestCarRentalSystem))

    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestB))
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestC))
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestD))
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestE))
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(TestF))
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)

# Call the function to run the tests
run_tests()

...............
----------------------------------------------------------------------
Ran 15 tests in 0.027s

OK
